<a href="https://colab.research.google.com/github/sibugma/predict-FOREX-with-ML-Logistic-Regression/blob/main/Forex-step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Download EURUSD exchange rate data
data = yf.download("EURUSD=X", start="2020-01-01", end="2023-01-01")

# Add technical indicators
# Simple Moving Average (SMA)
data['SMA'] = data['Close'].rolling(window=14).mean()

# Calculate RSI (Relative Strength Index)
delta = data['Close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
data['RSI'] = 100 - (100 / (1 + rs))

# Calculate MACD (Moving Average Convergence Divergence)
exp1 = data['Close'].ewm(span=12, adjust=False).mean()
exp2 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = exp1 - exp2
data['MACD_signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

# Add price change column and target
data['Price_Change'] = data['Close'].diff()
data['Target'] = np.where(data['Price_Change'] > 0, 1, 0)  # 1 if price increased, otherwise 0

# Drop NaN values
data.dropna(inplace=True)

# Define features and target variable
X = data[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA', 'RSI', 'MACD', 'MACD_signal']]
y = data['Target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on the test set and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Predict for new data (latest available data)
latest_data = data[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA', 'RSI', 'MACD', 'MACD_signal']].iloc[-1].values.reshape(1, -1)
prediction = model.predict(latest_data)
print("Prediction for the latest data:", "Up" if prediction[0] == 1 else "Down")


[*********************100%***********************]  1 of 1 completed

Accuracy: 0.538961038961039
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.53      0.54        80
           1       0.52      0.55      0.54        74

    accuracy                           0.54       154
   macro avg       0.54      0.54      0.54       154
weighted avg       0.54      0.54      0.54       154

Confusion Matrix:
 [[42 38]
 [33 41]]
Prediction for the latest data: Up
